- rimozione outlier
- rimozione errori (-1 su delta) --> NaN
- controllare che non ci siano righe con valori completmente nulli
- rimozione cobbled, gravel perchè sono tutti true e false
- rimuovere i ciclisti dal dataframe dei ciclisti che non hanno partecipato a gare ???
- rimozione delle feature correlate 
- drop temperatura

Columns containing null values in the cyclist dataset:
- birth_year
- weight
- height


Outliers in cyclist dataset:
- weight
- height

Columns containing null values in the races dataset:
- points
- uci_points
- climb_total
- profile
- cyclist age
- cyclist_team



Outliers in races dataset:
- points
- uci_points
- length
- climb_total
- start_list quality
- race_year
- cyclist_age
- delta







Duplicates in races dataset:
- cyclists
- 

Inconsistencies:
- cyclist_age
- delta

# Data Cleaning